In [ ]:
!pip install -U transformers
!pip install -U accelerate
!pip install -U bitsandbytes

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig

# quant cuz mem
# load in 4 bit is  Normal Float 4 (NF4) from the paper QLoRA
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# To prevent GPU memory
config = AutoConfig.from_pretrained("mistralai/Mistral-7B-v0.1")
config.gradient_checkpointing = True

# tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1",
                                             device_map = "cuda",
                                             trust_remote_code = True,
                                             quantization_config = quantization_config,
                                             config=config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [7]:
prompt = "Should I buy bitcoin"

model_inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True)

tokenizer.batch_decode(generated_ids)[0]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"<s> Should I buy bitcoin? No one can definitively answer that, but to make the right choice, it helps to understand what Bitcoin is, how it came to be, and where it's headed.\n\nIt is also a good idea to think about how you might use Bitcoin once you do decide to buy some.\n\nBitcoin is:\n\n●       A medium of exchange,\n\n●       A form of payment,\n\n●       An investment asset, and\n\n●       A store of value (like gold).\n\nWhat is Bitcoin?\n\nA medium of exchange\n\nIn order for people to sell and buy goods and services across the world, they need a common medium of exchange. They need to agree on what they will trade to each other to settle debts and make trades. Bitcoin allows people to easily and instantly trade and settle debts in a peer-to-peer fashion. A network of Bitcoin users can exchange their goods or services directly without incurring bank fees or needing to convert their currencies.\n\nA form of payment\n\nFor centuries, money has been used as a form of payment. Now